In [1]:
#DATA CLEANING AND PREPROCESSING

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [3]:
#loading and understanding the dataset
data = pd.read_csv("TA_restaurants_curated.csv")

FileNotFoundError: File b'TA_restaurants_curated.csv' does not exist

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data["Price Range"].head()

In [ ]:
Amsterdam = data[data.City == 'Amsterdam']
Amsterdam.head()

In [ ]:
data['Cuisine Style'].loc[30]

In [ ]:
city_count = data['City'].value_counts()
city_count.head()

In [ ]:
#number of missing values in the dataset
data.isna().sum()

In [ ]:
#renaming column 0
restaurant_data = data.rename(columns = {'Unnamed: 0':'Restaurant ID'})
restaurant_data.head()

In [ ]:
#filling missing values in Ranking
restaurant_data["Ranking"] = restaurant_data["Restaurant ID"] + 1

In [ ]:
restaurant_data.head()

In [ ]:
restaurant_data.isna().sum()

In [ ]:
na_loc = restaurant_data.index[restaurant_data['Number of Reviews'].isnull()]  #identifying the location of NANs

num_nas = len(na_loc)  #number of NANs

na_loc
num_nas

In [ ]:
#filling missing values in Number of Reviews

#import random
#restaurant_data['Number of Reviews'].fillna(random.random())
restaurant_data["Number of Reviews"].fillna(restaurant_data['Number of Reviews'].mean(), inplace=True)
restaurant_data["Number of Reviews"] = restaurant_data['Number of Reviews'].round(0)

In [ ]:
restaurant_data['Number of Reviews'].head()

In [ ]:
restaurant_data.head()

In [ ]:
#missing values in Price Range
restaurant_data.groupby('City')['Price Range'].value_counts()   

In [ ]:
restaurant_data['Price Range'].fillna(restaurant_data['Price Range'].value_counts().index[0],inplace=True)

In [ ]:
restaurant_data.isna().sum()

In [ ]:
#Converting price range into numerical values
from sklearn.preprocessing import LabelEncoder

number = LabelEncoder()
restaurant_data['Price Range'] = number.fit_transform(restaurant_data['Price Range'].astype('str'))

In [ ]:
restaurant_data['Price Range'].head(3)

In [ ]:
restaurant_price = pd.get_dummies(restaurant_data['Price Range'])

In [ ]:
type(restaurant_price)

In [ ]:
restaurants = pd.concat([restaurant_data,restaurant_price], axis=1)

In [ ]:
restaurants.head()

In [ ]:
price_rename = restaurants.rename(columns = {0:'Price Range_$',1:'Price Range_$$-$$$',2:'Price range_$$$$'})

In [ ]:
price_rename.head()

In [ ]:
restaurants1 = price_rename.drop(['Price Range'], axis=1)
restaurants1.head()

In [ ]:
restaurants1.isna().sum()

In [ ]:
#drop missing review rows
#restaurants2 = restaurants1.dropna(subset=['Reviews'])

In [ ]:
#missing review values
restaurants1['Reviews'] = restaurants1['Reviews'].fillna('["No Review"]', axis=0)

In [ ]:
restaurants1.isna().sum()

In [ ]:
restaurants1['Reviews'].tail()

In [ ]:
restaurants1['Reviews'][3233]

In [ ]:
restaurants1['Reviews'] = restaurants1['Reviews'].replace(['[[], []]'], 'No Review')

In [ ]:
restaurants1['Reviews']

In [ ]:
restaurants1['Reviews'][3233]

In [ ]:
len(restaurants1)

In [ ]:
"Vietnames" in restaurants1['Cuisine Style']

In [ ]:
type(restaurants1['Cuisine Style'][175])

In [ ]:
restaurants1['Cuisine Style'] = restaurants1['Cuisine Style'].fillna('["Unknown"]', axis=0)

In [ ]:
restaurants1.tail()

In [ ]:
restaurants1['Cuisine Style'][175]

In [ ]:
restaurants1['Cuisine Style'][3243]

In [ ]:
len(restaurants1['Cuisine Style'])

In [ ]:
restaurants1.isna().sum()

In [ ]:
restaurants1['Cuisine Style'][125189]

In [ ]:
#vectorization (dealing with the missing cuisine style values)
cuisine_style = []

for i in range(len(restaurants1['Cuisine Style'])):
    word = restaurants1['Cuisine Style'].loc[i]
    j = word.replace("'","").replace("[","").replace("]","").replace("\"","")   #escape sequence("\"")
    j = j.replace(" ", "")
    print(j,i)
    y = j.split(',')
    cuisine_style.append(y)

cuisine_style

In [ ]:
len(cuisine_style)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

df = pd.DataFrame({'Cuisine Style':cuisine_style}, columns=['Cuisine Style'])
transform_df = df['Cuisine Style']
mlb = MultiLabelBinarizer()
cuisines = pd.DataFrame(mlb.fit_transform(transform_df),columns=mlb.classes_, index=df.index)

In [ ]:
cuisines

In [ ]:
cuisines.loc[0,:]

In [ ]:
#concatenate the cuisines 
restaurants2 = pd.concat([restaurants1,cuisines], axis=1)
restaurants2

In [ ]:
list(restaurants2)

In [ ]:
#drop cuisine style column
restaurants3 = restaurants2.drop(['Cuisine Style'], axis=1)
restaurants3

In [ ]:
#missing rating values
restaurants3.isna().sum()

In [ ]:
restaurants3.Rating.value_counts()

In [ ]:
restaurants3.Rating.unique()

In [ ]:
restaurants3["Rating"].fillna(restaurants3['Rating'].mean(), inplace=True)

In [ ]:
for col in ['Rating']:
    restaurants3[col] = restaurants3[col].astype(int)

In [ ]:
restaurants3

In [ ]:
restaurants3.Rating.value_counts()

In [ ]:
restaurants3.Rating.unique()

In [ ]:
#sentiments associated with ratings

#terrible = [0]

#bad = [1]

#average = [2]

#good = [3]

#very good = [4]

#excellent = [5]

In [ ]:
#working with the text data in reviews - Sentiment Analysis

In [ ]:
sentiment_data = restaurants3[['Rating', 'Reviews']]
sentiment_data

In [ ]:
sdata = sentiment_data.values    #gives an numpy array
sdata                   

In [ ]:
len(sdata)

In [ ]:
count = len(sdata)
all_data = []

stop1 = int(0.25 * count)     #creating batches
stop2 = int(0.5 * count)
stop3 = int(0.75 * count)

In [ ]:
stop1, stop2, stop3, count    #batches of data points

In [ ]:
for i in range(count):
    rating = sdata[i, 0]
    reviews = sdata[i, 1].split('], [')[0]     #spliting the reviews and date strings from a single list and considering only the reviews
    reviews = reviews.replace("[[", "")
    reviews = reviews.replace("'", "")
    reviews = reviews.replace('"', '')
    reviews = reviews.split(',')
    print(reviews)
    for review in reviews:
        all_data.append([review, rating])

In [ ]:
sent_data = pd.DataFrame(all_data, columns=['Review', 'Rating'])

In [ ]:
sent_data

In [ ]:
#remove the puntuation marks
sent_data['Review'] = sent_data['Review'].str.replace('[^\w\s]','')
sent_data.head(3)

In [ ]:
#splitting sentences into list of words  
tokenized_data = sent_data['Review'].apply(lambda x : x.lower().split())    
tokenized_data.head(5)

In [ ]:
import nltk      #natural language toolkit
from nltk.stem.porter import *

In [ ]:
stemmer = PorterStemmer()
stem_data = tokenized_data.apply(lambda x: [stemmer.stem(i) for i in x])  # stemming

In [ ]:
stem_data

In [ ]:
stemmed_data = []
for i in range(len(stem_data)):
    stemmed_data.append(' '.join(stem_data[i]))

stemmed_data

In [ ]:
np.array(stemmed_data).reshape(-1,1)

In [ ]:
sent_data['Cleaned_Review'] = np.array(stemmed_data).reshape(-1,1)

In [ ]:
sent_data

In [ ]:
#splitting train/test data

textReviews = sent_data['Cleaned_Review']
Y = sent_data['Rating']

In [ ]:
Y = np.array(Y)

In [ ]:
for i in range(len(Y)):
    Y[Y < 0] = 0

In [ ]:
Y

In [ ]:
textReviews_train, textReviews_test, Y_train, Y_test = train_test_split(textReviews, Y, # data we want to split 
                                            train_size = 0.7,
                                            random_state = 500, # shuffle rows
                                            stratify = Y) # ensure classes the same in train/test

In [ ]:
#change text data into numerical for the classifer to understand
#We will use a bag of words model. To break up our 'Customer reviews' sentences into words (tokens) with the Count Vectorizer.

In [ ]:
#we call the sklearn count Vectorizer (transformer) to transform
#text into a vector
from sklearn.feature_extraction.text import CountVectorizer 
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=10000, stop_words='english') 

In [ ]:
#use vetorizer var to fit on text
#bag-of-words feature matrix
X_train = bow_vectorizer.fit_transform(textReviews_train.values)
X_test = bow_vectorizer.fit_transform(textReviews_test.values)

In [ ]:
print(X_train.shape)
print(X_train.toarray())

In [ ]:
print(X_test.shape)
print(X_test.toarray())

In [ ]:
# use vocabulary_ to see words in the vocabulary
vocabulary = bow_vectorizer.vocabulary_
vocabulary

In [ ]:
#dimensionality reduction using truncated singular value decomposition (SVD)

from sklearn.decomposition import TruncatedSVD

tsvd = TruncatedSVD(n_components=500)  
x_train = tsvd.fit_transform(X_train)
x_test = tsvd.fit_transform(X_test)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

In [ ]:
y_train = pd.get_dummies(Y_train).values
y_train

In [ ]:
y_test = pd.get_dummies(Y_test).values
y_test

In [ ]:
#model training using keras

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
x_train.shape

In [ ]:
#model architecture
model = Sequential([   
    Dense(729, activation = 'relu', input_shape = (x_train[1].shape)),
    Dense(389, activation="relu"),
    Dense(153, activation='relu'),
    Dense(6, activation = 'softmax')
])

In [ ]:
model.summary()

In [ ]:
adam = keras.optimizers.adam(lr = 0.01)

model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = adam,
              metrics = ['accuracy'])

In [ ]:
history = model.fit(x_train, Y_train, epochs = 40, verbose = 1)

In [ ]:
history = model.fit(x_train, Y_train, epochs = 40, batch_size = 1024, verbose = 1)